# サンプルコードの日本語解説
　1. 目的  
     **機械学習はじめたてで、サンプルみても何してるかさっぱりわからん。  
     1行毎に日本語で解説いれて理解する。ライブラリの中身も何してるかわかるようにする。**
     
  2. 対象
    - [解説対象コード](https://github.com/keras-team/keras/blob/master/examples/cifar10_cnn.py)
    
  3. これなに？
      - Trains a simple deep CNN on the CIFAR10 small images dataset.
      - 直訳：CIFAR10の小さな画像データセットでシンプルな深いCNNを学習します。

# CIFAR10ってなに？
 - ラベルづけしためっちゃいっぱいの画像データを公開してるところ。画像データといっても犬の画像そのままじゃくて、ピクセルデータになったやつが公開されてる。あるピクセルデータに犬とか猫とか正解が付いた状態。便利！
 - [CIFAR10](https://www.cs.toronto.edu/~kriz/cifar.html)

# CNNってなに？
 - Convolution Neural Networkの略
 - 機械学習のアルゴリズムの一種。画像認識で実績がある。

# ライブラリ群のインポート

In [21]:
#coding: utf-8
from __future__ import print_function
# Kerasはニューラルネットワークライブラリ
# TensorFlowやTheano上で動く。
import keras
# 画像データセット
from keras.datasets import cifar10
# 画像データの拡張するためのクラス
from keras.preprocessing.image import ImageDataGenerator
# Sequentialモデルクラス
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import os
import pprint

# ライブラリに使うパラメータ設定

In [13]:
# 勾配更新毎のサンプル数を示す整数
batch_size = 32
# 画像データの種類
num_classes = 10
# 訓練データ配列の反復回数を示す整数
epochs = 100
data_augmentation = True
# モデル保存パス
save_dir = os.path.join(os.getcwd(), 'saved_models')

# データセットの取得

In [31]:
model_name = 'keras_cifar10_trained_model.h5'

# The data, split between train and test sets:
# cifar10から訓練用と検証用データセットに分けたものを取得。
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
# 訓練用データの詳細
# 入力値　N数：5万　width:32 height:32 channel:3
print('x_train shape:', x_train.shape)
print('x_train dimension: ', x_train.ndim)
print('y_train shape:', y_train.shape)
print('y_train dimension: ', y_train.ndim)
# 検証用データの詳細
# 入力値　N数：1万　width:32 height:32 channel:3
print('x_test shape:', x_test.shape)
print('x_test dimension: ', x_test.ndim)
print('y_test shape:', y_test.shape)
print('y_test dimension: ', y_test.ndim)

x_train shape: (50000, 32, 32, 3)
x_train dimension:  4
y_train shape: (50000, 1)
y_train dimension:  2
x_test shape: (10000, 32, 32, 3)
x_test dimension:  4
y_test shape: (10000, 1)
y_test dimension:  2


# ニューラルネットワークの構築

In [15]:
# Convert class vectors to binary class matrices.
# 整数クラスベクトルから2価クラスの行列への変換
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print('y_train shape:', y_train.shape)
print('y_train dimension: ', y_train.ndim)
print('y_test shape:', y_test.shape)
print('y_test dimension: ', y_test.ndim)

y_train shape: (50000, 10)
y_train dimension:  2
y_test shape: (10000, 10)
y_test dimension:  2


In [ ]:
# モデルの宣言
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        zca_epsilon=1e-06,  # epsilon for ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        # randomly shift images horizontally (fraction of total width)
        width_shift_range=0.1,
        # randomly shift images vertically (fraction of total height)
        height_shift_range=0.1,
        shear_range=0.,  # set range for random shear
        zoom_range=0.,  # set range for random zoom
        channel_shift_range=0.,  # set range for random channel shifts
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        cval=0.,  # value used for fill_mode = "constant"
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False,  # randomly flip images
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None,
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0.0)

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        epochs=epochs,
                        validation_data=(x_test, y_test),
                        workers=4)

# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])